# Cargar CSVs (uno-a-muchos) a Azure AI Search
En este Jupyter Notebook, creamos y ejecutamos pasos para indexar un archivo CSV en el que cada fila es un registro/documento individual e independiente. A continuación, cada fila se puede buscar en Azure AI Search. 
La documentación de referencia se puede encontrar en [Indexing blobs and files to produce multiple search documents](https://learn.microsoft.com/en-us/azure/search/search-howto-index-one-to-many-blobs).

Por defecto, un indexador tratará el contenido de un blob o archivo como un único documento de búsqueda. Si deseas una representación más granular en un índice de búsqueda, puedes establecer valores `parsingMode` para crear múltiples documentos de búsqueda a partir de un blob o archivo.

Vamos a utilizar la misma cuenta privada de Blob Storage pero un contenedor diferente que contiene resúmenes de 90k publicaciones médicas sobre COVID-19 publicadas en 2020-2022. Este archivo es un subconjunto de un conjunto de datos mucho mayor (770k artículos) llamado CORD19 [AQUÍ](https://github.com/allenai/cord19)

In [1]:
import os
import json
import requests
from dotenv import load_dotenv
load_dotenv("credentials.env")

# Name of the container in your Blob Storage Datasource ( in credentials.env)
BLOB_CONTAINER_NAME = "cybersecurity"

In [2]:
# Define the names for the data source, index and indexer
datasource_name = "cogsrch-datasource-kiografia-csv"
skillset_name = "cogsrch-skillset-kiografia-csv"
index_name = "cogsrch-index-kiografia-csv"
indexer_name = "cogsrch-indexer-kiografia-csv"

In [3]:
# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}
params = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']}

## Crear fuente de datos (contenedor Blob con el archivo de datos CSV CORD19)

In [4]:
# Create a data source

datasource_payload = {
    "name": datasource_name,
    "description": "Demo files to demonstrate cognitive search capabilities of one-to-many.",
    "type": "azureblob",
    "credentials": {
        "connectionString": os.environ['BLOB_CONNECTION_STRING']
    },
    "container": {
        "name": BLOB_CONTAINER_NAME
        # "query": "cord192"
    }
}
r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/datasources/" + datasource_name,
                 data=json.dumps(datasource_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

204
True


## Inspeccione el archivo CSV para que podamos entender los tipos de columnas antes de crear el índice

En nuestro conjunto de datos privado hemos colocado una versión más pequeña del archivo original metadata.csv en el conjunto de datos cord19. 
Veamos qué aspecto tiene el archivo:

In [ ]:
#Download the csv files to disk and inspect using pandas
import pandas as pd
#https://blobstoragedorak.blob.core.windows.net/cybersecurity/Ventas_SalesForce.xlsx%20-%20Info%20de%20ventas.csv
#https://blobstoragedorak.blob.core.windows.net/cybersecurity/InfoClientes_SOC.xlsx%20-%20Info%20de%20SOC.csv
#https://blobstoragedorak.blob.core.windows.net/cybersecurity/Tickets_ServiceDesk.xlsx%20-%20Info%20de%20tickets.csv
remote_file_path = "https://blobstoragedorak.blob.core.windows.net/cybersecurity/Tickets_ServiceDesk.xlsx%20-%20Info%20de%20tickets.csv"

In [ ]:
metadata = pd.read_csv(remote_file_path + os.environ['BLOB_SAS_TOKEN'])
#metadata.to_csv('metadata.csv', index=False)
print("No. of lines:",metadata.shape[0])

simple_schema = ['cord_uid', 'source_x', 'title', 'abstract', 'authors', 'url']

def make_clickable(address):
    '''Make the url clickable'''
    return '<a href="{0}">{0}</a>'.format(address)

def preview(text):
    '''Show only a preview of the text data.'''
    return text[:30] + '...'

format_ = {'title': preview, 'abstract': preview, 'authors': preview, 'url': make_clickable}

metadata[simple_schema].head().style.format(format_)

## Creación del índice
En Azure AI Search, tanto los indexadores de blob como los indexadores de archivos admiten un modo de análisis sintáctico de texto delimitado para archivos CSV que trata cada línea del CSV como un documento de búsqueda independiente.
### **Importante**:
Como puede ver a continuación y en el cuaderno anterior, hay 6 campos obligatorios en el esquema: `id, title, name, location, chunk, chunkVector`. Estos campos deben existir en cualquier índice que cree independientemente de la fuente de datos. Cualquier campo adicional es bueno añadirlo para que el motor pueda buscar documentos relevantes, sin embargo los campos obligatorios deben existir para que todo el código funcione sin problemas.

In [7]:
# Create an index
# Queries operate over the searchable fields and filterable fields in the index
index_payload = {
    "name": index_name,
    "vectorSearch": {
        "algorithms": [
            {
                "name": "myalgo",
                "kind": "hnsw"
            }
        ],
        "vectorizers": [
            {
                "name": "openai",
                "kind": "azureOpenAI",
                "azureOpenAIParameters":
                {
                    "resourceUri" : os.environ['AZURE_OPENAI_ENDPOINT'],
                    "apiKey" : os.environ['AZURE_OPENAI_API_KEY'],
                    "deploymentId" : os.environ['EMBEDDING_DEPLOYMENT_NAME']
                }
            }
        ],
        "profiles": [
            {
                "name": "myprofile",
                "algorithm": "myalgo",
                "vectorizer":"openai"
            }
        ]
    },
    "semantic": {
        "configurations": [
            {
                "name": "my-semantic-config",
                "prioritizedFields": {
                    "titleField": {
                        "fieldName": "cliente"
                    },
                    "prioritizedContentFields": [
                        {
                            "fieldName": "chunk"
                        }
                    ],
                    "prioritizedKeywordsFields": []
                }
            }
        ]
    },
    "fields": [
        {"name": "id", "type": "Edm.String", "key": "true", "analyzer": "keyword", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false","facetable": "false"},
        {"name": "ParentKey", "type": "Edm.String", "searchable": "true", "retrievable": "true", "facetable": "false", "filterable": "true", "sortable": "false"},
        {"name": "title", "type": "Edm.String", "searchable": "true", "retrievable": "true", "facetable": "false", "filterable": "true", "sortable": "false"},
        {"name": "name", "type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "location", "type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},   
        {"name": "chunk","type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {
            "name": "chunkVector",
            "type": "Collection(Edm.Single)",
            "dimensions": 1536,   # IMPORTANT: Make sure these dimmensions match your embedding model name
            "vectorSearchProfile": "myprofile",
            "searchable": "true",
            "retrievable": "true",
            "filterable": "false",
            "sortable": "false",
            "facetable": "false"
        }
    ]
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + index_name,
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)
print(r.text)


400
False
{"error":{"code":"InvalidRequestParameter","message":"The request is invalid. Details: definition : Unknown field 'cliente' in semantic field list.","details":[{"code":"UnknownField","message":"Unknown field 'cliente' in semantic field list. Parameters: definition"}]}}


##  Crear Skillset - Text Splitter, Language Detection
No necesitamos usar la skill OCR y Merge, ya que sabemos con seguridad que esto es solo parseo de texto. Solo usaremos el SplitSkill y el AzureOpenAIEmbeddingSkill.

In [ ]:
"""
cliente
ID de la solicitud
Asunto
Técnico
Grupo de soporte asignado
Producto
Categoría 3
Fecha de creación de ticket
Fecha de cerrado de ticket
Estado de solicitud
Creado por
Incidente de seguridad
Impacto
Urgencia
Prioridad
Estado vencido
Código de cierre de solicitud
Auxiliar
Tiempo de resolución en horas
Tipo de Incidencia
Tipo de ticket
Notificación

"""

In [15]:
# Create a skillset
skillset_payload = {
    "name": skillset_name,
    "description": "e2e Skillset for RAG - One to Many",
    "skills":
    [
        {
            "@odata.type": "#Microsoft.Skills.Text.SplitSkill",
            "context": "/document",
            "textSplitMode": "pages",
            "maximumPageLength": 5000, # 5000 is default
            "defaultLanguageCode": "en",
            "inputs": [
                {
                    "name": "text",
                    "source": "/document/content"
                }
            ],
            "outputs": [
                {
                    "name": "textItems",
                    "targetName": "chunks"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.AzureOpenAIEmbeddingSkill",
            "description": "Azure OpenAI Embedding Skill",
            "context": "/document/chunks/*",
            "resourceUri": os.environ['AZURE_OPENAI_ENDPOINT'],
            "apiKey": os.environ['AZURE_OPENAI_API_KEY'],
            "deploymentId": os.environ['EMBEDDING_DEPLOYMENT_NAME'],
            "inputs": [
                {
                    "name": "text",
                    "source": "/document/chunks/*"
                }
            ],
            "outputs": [
                {
                    "name": "embedding",
                    "targetName": "vector"
                }
            ]
        }
    ],
    "indexProjections": {
        "selectors": [
            {
                "targetIndexName": index_name,
                "parentKeyFieldName": "ParentKey",
                "sourceContext": "/document/chunks/*",
                "mappings": [
                    {
                        "name": "title",
                        "source": "/document/title"
                    },
                    {
                        "name": "name",
                        "source": "/document/name"
                    },
                    {
                        "name": "location",
                        "source": "/document/location"
                    },
                    {
                        "name": "chunk",
                        "source": "/document/chunks/*"
                    },
                    {
                        "name": "chunkVector",
                        "source": "/document/chunks/*/vector"
                    }
                ]
            }
        ],
        "parameters": {
            "projectionMode": "skipIndexingParentDocuments"
        }
    },
    "cognitiveServices": {
        "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
        "description": os.environ['COG_SERVICES_NAME'],
        "key": os.environ['COG_SERVICES_KEY']
    }
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/skillsets/" + skillset_name,
                 data=json.dumps(skillset_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

204
True


## Crear y ejecutar el indexador - (ejecuta el pipeline)
Para crear indexadores de uno a varios con blobs CSV, cree o actualice una definición de indexador con el modo de análisis sintáctico delimitedText

In [11]:
indexer_payload = {
    "name": indexer_name,
    "dataSourceName": datasource_name,
    "targetIndexName": index_name,
    "skillsetName": skillset_name,
    #"schedule" : { "interval" : "PT30M"}, # How often do you want to check for new content in the data source
    "fieldMappings": [
        {
          "sourceFieldName" : "metadata_storage_name",
          "targetFieldName" : "name"
        },
        {
          "sourceFieldName" : "url",
          "targetFieldName" : "location"
        }
    ],
    "outputFieldMappings":
    [],
    "parameters" : { 
        "configuration" : { 
            "dataToExtract": "contentAndMetadata",
            "parsingMode" : "delimitedText", 
            "firstLineContainsHeaders" : True,
            "delimitedTextDelimiter": ","
        } 
    }
}
r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexers/" + indexer_name,
                 data=json.dumps(indexer_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

204
True


In [13]:
print(r.text)

In [14]:
# Optionally, get indexer status to confirm that it's running
try:
    r = requests.get(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexers/" + indexer_name +
                     "/status", headers=headers, params=params)
    # pprint(json.dumps(r.json(), indent=1))
    print(r.status_code)
    print("Status:",r.json().get('lastResult').get('status'))
    print("Items Processed:",r.json().get('lastResult').get('itemsProcessed'))
    print(r.ok)
    
except Exception as e:
    print("Wait a few seconds until the process starts and run this cell again.")

200
Status: inProgress
Items Processed: 0
True


**Cuando el indexador termine de funcionar, tendremos las 90.000 filas indexadas correctamente como documentos separados en nuestro motor de búsqueda.**


# Referencia

- https://learn.microsoft.com/en-us/azure/search/search-howto-index-csv-blobs



# SIGUIENTE
Ahora que tenemos dos índices separados cargados con dos tipos diferentes de información, En el siguiente cuaderno 3, vamos a hacer una consulta Multi-Index, ordenar los resultados en función de la puntuación semántica reranker de Azure AI Search, y luego usar OpenAI para entender los resultados y dar la mejor respuesta posible